# RAG

A continuación se muestra una descripción general de alto nivel del sistema que queremos construir:


<img src='images/img_1.png' width="800">

# PARTE I

Empecemos cargando las variables de entorno que necesitamos utilizar.

## Setting up the model
Definamos el modelo LLM que utilizaremos como parte del flujo de trabajo.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Este es el video de YouTube que vamos a utilizar.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=F8NKVhkZZWI&t=1s"

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

Probamos el modelo haciendo una pregunta sencilla

In [ ]:
pregunta_sencilla = "¿Cuál es la capital de Túnez?"
respuesta = model.invoke(pregunta_sencilla)

# Imprimimos el contenido de la respuesta
print(respuesta.content)

La capital de Túnez es Túnez.


El resultado del modelo es una instancia de `AIMessage` que contiene la respuesta. Podemos extraer esta respuesta encadenando el modelo con un analizador de salida [outputParser](https://python.langchain.com/docs/modules/model_io/output_parsers/).

Así es como se ve el encadenamiento del modelo con un analizador de salida:

<img src='images/chain1.png' width="1200">

Para este ejemplo, utilizaremos un `StrOutputParser` simple para extraer la respuesta como una cadena.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

#(convertir AIMessage a string)
parser = StrOutputParser()

# Creamos la cadena simple combinando el modelo y el parser
chain = model | parser

# Probamos la cadena con la misma pregunta sencilla
pregunta_sencilla = "¿Cuál es la capital de Alemaña?"
respuesta_parseada = chain.invoke(pregunta_sencilla)

# Imprimimos la respuesta (ahora debería ser una cadena directamente)
print(respuesta_parseada)
print(type(respuesta_parseada)) # Para verificar que es un string

La capital de Alemania es Berlín.
<class 'str'>


## Presentamos las plantillas de preguntas

Queremos contextualizar el modelo y la pregunta. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) Son una forma sencilla de definir y reutilizar indicaciones.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Responda la pregunta según el contexto descrito a continuación. Si no puede responder, responda "No lo sé".

Contexto: {contexto}

Pregunta: {pregunta}
"""


Ahora podemos encadenar el mensaje con el modelo y el analizador de salida.

<img src='images/chain2.png' width="1200">

In [ ]:
from langchain_core.runnables import  RunnablePassthrough

chain = (
         {
            "contexto": RunnablePassthrough(),
            "pregunta": RunnablePassthrough()
        }
        | ChatPromptTemplate.from_template(template)
        | model
        | parser
    )

## Combinación de cadenas

Podemos combinar diferentes cadenas para crear flujos de trabajo más complejos. Por ejemplo, creemos una segunda cadena que traduzca la respuesta de la primera a otro idioma.

Comencemos creando una nueva plantilla de solicitud para la cadena de traducción:

In [ ]:
translation_prompt = ChatPromptTemplate.from_template(
    "Traduce {answer} al {language}"
)

Ahora podemos crear una nueva cadena de traducción que combine el resultado de la primera cadena con la solicitud de traducción.

Así es como se ve el nuevo flujo de trabajo:

<img src='images/chain3.png' width="1200">

In [ ]:
from operator import itemgetter

# Cadena de respuesta usando contexto (primera cadena, ya definida anteriormente)
qa_chain = (
    {
        "contexto": RunnablePassthrough(),
        "pregunta": RunnablePassthrough()
    }
    | ChatPromptTemplate.from_template(template)
    | model
    | parser
)

# Crear la cadena de traducción
translation_chain = (
    {
        "answer": RunnablePassthrough(),
        "language": lambda _: "Inglés"  # valor predeterminado
    }
    | translation_prompt
    | model
    | parser
)

# Combinamos ambas cadenas: primero obtenemos la respuesta, luego la traducimos
combined_chain = qa_chain | (lambda answer: translation_chain.invoke({"answer": answer, "language": "Castellano"}))

# Probemos la cadena combinada
respuesta_traducida = combined_chain.invoke({
    "contexto": "París es la capital de Francia y una de las ciudades más visitadas del mundo.",
    "pregunta": "¿Cuál es la capital de Francia?"
})

print("Respuesta traducida:", respuesta_traducida)

Respuesta traducida: Translate {'answer': 'The capital of France is Paris.', 'language': 'Spanish'} into English


# PARTE II

## Transcripcion de video de YouTube

El contexto que queremos enviar al modelo proviene de un video de YouTube. Descargamos el video y transcribámoslo con [OpenAI's Whisper](https://openai.com/research/whisper).

In [ ]:
import tempfile
import whisper
import os
import yt_dlp  # Using yt-dlp instead of pytube

if not os.path.exists("transcription.txt"):
    print(f"Descargando video: {YOUTUBE_VIDEO}")
    
    # Create a temporary directory for the download
    with tempfile.TemporaryDirectory() as tmpdir:
        # yt-dlp options for downloading audio only
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(tmpdir, 'audio.%(ext)s'),
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': False
        }
        
        # Download the audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.extract_info(YOUTUBE_VIDEO, download=True)
            audio_file = os.path.join(tmpdir, 'audio.mp3')
        
        print(f"Transcribiendo archivo de audio: {audio_file}")
        
        # Load Whisper model
        whisper_model = whisper.load_model("base")
        
        # Transcribir el audio
        transcription = whisper_model.transcribe(audio_file, fp16=False)["text"].strip()
        
        # Save the transcription to a file
        with open("transcription.txt", "w") as file:
            file.write(transcription)
        
        print("Transcription completed and saved to 'transcription.txt'")
else:
    print("Transcription file already exists!")

Descargando video: https://www.youtube.com/watch?v=F8NKVhkZZWI&t=1s
[youtube] Extracting URL: https://www.youtube.com/watch?v=F8NKVhkZZWI&t=1s
[youtube] F8NKVhkZZWI: Downloading webpage
[youtube] F8NKVhkZZWI: Downloading tv client config
[youtube] F8NKVhkZZWI: Downloading player 20830619
[youtube] F8NKVhkZZWI: Downloading tv player API JSON
[youtube] F8NKVhkZZWI: Downloading ios player API JSON
[youtube] F8NKVhkZZWI: Downloading m3u8 information
[info] F8NKVhkZZWI: Downloading 1 format(s): 251
[download] Destination: /var/folders/fl/0425_ksd7gl2jnqxhr8qwwhw0000gn/T/tmp5c52usnh/audio.webm
[download] 100% of    9.45MiB in 00:00:00 at 16.81MiB/s    
[ExtractAudio] Destination: /var/folders/fl/0425_ksd7gl2jnqxhr8qwwhw0000gn/T/tmp5c52usnh/audio.mp3
Deleting original file /var/folders/fl/0425_ksd7gl2jnqxhr8qwwhw0000gn/T/tmp5c52usnh/audio.webm (pass -k to keep)
Transcribiendo archivo de audio: /var/folders/fl/0425_ksd7gl2jnqxhr8qwwhw0000gn/T/tmp5c52usnh/audio.mp3
Transcription completed and s

Vamos a leer la transcripción y mostrar los primeros caracteres para asegurarnos de que todo funciona como se espera.

In [ ]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]

'2024 will be the year of AI agents. So what are AI agents? To start explaining that, we have to look'

## Usando la transcripción completa como contexto

Si intentamos invocar la cadena usando la transcripción como contexto, el modelo devolverá un error porque el contexto es demasiado largo.

Los modelos de lenguaje grandes admiten tamaños de contexto limitados. El vídeo que estamos usando es demasiado largo para que el modelo lo pueda procesar, por lo que necesitamos buscar una solución diferente.

In [ ]:
# Primero, aseguramos que tengamos la transcripción cargada
with open("transcription.txt") as file:
    transcription = file.read()

# Ahora intentamos usar la transcripción como contexto
try:
    chain.invoke({
        "context": transcription,
        "question": "¿Es una buena idea leer artículos?"
    })
except Exception as e:
    print(e)

## División de la transcripción

Dado que no podemos usar la transcripción completa como contexto para el modelo, una posible solución es dividir la transcripción en fragmentos más pequeños. Así, podemos invocar el modelo utilizando solo los fragmentos relevantes para responder a una pregunta específica:

<img src='images/system2.png' width="1200">

Comencemos cargando la transcripción en la memoria:

In [ ]:
from langchain_community.document_loaders import TextLoader

# Load the transcription file
loader = TextLoader("transcription.txt")
documents = loader.load()

# Print basic info about the loaded document
print(f"Loaded {len(documents)} document")
print(f"Text length: {len(documents[0].page_content)} characters")



Loaded 1 document
Text length: 10626 characters


Hay muchas maneras de dividir un documento. En este ejemplo, usaremos un divisor simple que divide el documento en fragmentos de tamaño fijo. Consulta [Divisores de texto](https://python.langchain.com/docs/modules/data_connection/document_transformers/) para obtener más información sobre los diferentes enfoques para dividir documentos.

A modo de ejemplo, dividiremos la transcripción en fragmentos de 100 caracteres con una superposición de 20 caracteres y mostraremos los primeros fragmentos:

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# First load the document
loader = TextLoader("transcription.txt")
documents = loader.load()

# Create a text splitter with chunk size of 100 and overlap of 20 characters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

# Split the document into chunks
chunks = text_splitter.split_documents(documents)

# Display information about the chunks
print(f"Split the document into {len(chunks)} chunks")

# Show the first 3 chunks as an example
print("\nFirst three chunks:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1}:")
    print(f"Length: {len(chunk.page_content)} characters")
    print(f"Content: {chunk.page_content}")

Split the document into 133 chunks

First three chunks:

Chunk 1:
Length: 100 characters
Content: 2024 will be the year of AI agents. So what are AI agents? To start explaining that, we have to look

Chunk 2:
Length: 95 characters
Content: we have to look at the various ships that we're seeing in the field of generative AI. The first

Chunk 3:
Length: 91 characters
Content: AI. The first ship I would like to talk to you about is this move from monolithic models to


Para nuestra aplicación específica, utilizaremos 1000 caracteres en su lugar:

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a text splitter with chunk size of 1000 and overlap of 200 characters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# Split the document into chunks
chunks = text_splitter.split_documents(documents)

# Display information about the chunks
print(f"Split the document into {len(chunks)} chunks")

# Show the first chunk as an example
if chunks:
    print("\nFirst chunk:")
    print(f"Length: {len(chunks[0].page_content)} characters")
    print(f"Content: {chunks[0].page_content}")

Split the document into 14 chunks

First chunk:
Length: 998 characters
Content: 2024 will be the year of AI agents. So what are AI agents? To start explaining that, we have to look at the various ships that we're seeing in the field of generative AI. The first ship I would like to talk to you about is this move from monolithic models to compound AI systems. So models on their own are limited by the data they've been trained on. So that impacts what they know about the world and what sort of tasks they can solve. They are also hard to adapt. So you could tune a model, but it would take an investment in data and in resources. So let's take a concrete example to illustrate this point. I want to plan a vacation for this summer, and I want to know how many vacation days are at my disposal. What I can do is take my query, feed that into a model that can generate a response. I think we can all expect that this answer will be incorrect because the model doesn't know who I am and does not have 

# PARTE III

## Configuración de un Vector Store

Necesitamos una forma eficiente de almacenar fragmentos de documentos, sus Embeddings y realizar búsquedas de similitud a gran escala. Para ello, usaremos un Vector Store.

Un Vector Store es una base de datos de Embeddings especializada en búsquedas rápidas de similitud.


<img src='images/chain4.png' width="1200">

Necesitamos configurar un retriever (https://python.langchain.com/docs/how_to/#retrievers). Este retriever realizará una búsqueda de similitud en el almacén vectorial y devolverá los documentos más similares al siguiente paso de la cadena.

## Configurar Pinecone

Para este ejemplo, usaremos [Pinecone](https://www.pinecone.io/).

<img src="images/pinecone.png" width="800">

El primer paso es crear una cuenta de Pinecone, configurar un índice, obtener una clave API y configurarla como variable de entorno `PINECONE_API_KEY`.

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Create Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create the index name 
index_name = "rag-transcription"

# Si el índice existe, lo eliminamos primero para recrearlo con la dimensión correcta
if index_name in pc.list_indexes().names():
    print(f"Eliminating existing index '{index_name}' to recreate with correct dimensions")
    pc.delete_index(index_name)
    # Esperar un momento para que la eliminación se complete
    import time
    time.sleep(5)

# Create the index with the correct dimensions
pc.create_index(
    name=index_name,
    dimension=384,  # HuggingFace 'all-MiniLM-L6-v2' embeddings have 384 dimensions
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
print(f"Created new index '{index_name}' with dimension 384")

# Initialize HuggingFace embeddings model (no API key needed, runs locally)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the vector store and load the documents
vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,  # Use your previously created chunks
    embedding=embeddings,
    index_name=index_name
)

print(f"Successfully loaded {len(chunks)} chunks into Pinecone index '{index_name}'")

Created new index 'rag-transcription' with dimension 384


/var/folders/fl/0425_ksd7gl2jnqxhr8qwwhw0000gn/T/ipykernel_87365/1861014624.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


Successfully loaded 14 chunks into Pinecone index 'rag-transcription'


Ahora ejecutemos una búsqueda de similitud en pinecone para asegurarnos de que todo funciona:

In [ ]:
# Test similarity search
query = "What are the main topics discussed in the video?"
docs = vectorstore.similarity_search(query, k=3)

print("\nResults for query:", query)
print("-" * 50)
for i, doc in enumerate(docs, 1):
    print(f"\nResult {i}:")
    print(doc.page_content)
    print("-" * 50)


Results for query: What are the main topics discussed in the video?
--------------------------------------------------

Result 1:
of system design with agentic behavior. And of course, you'll have a human in the loop, in most cases, as the accuracy is improving. I hope you found this video very useful. And please subscribe to the channel to learn more.
--------------------------------------------------

Result 2:
question they've been asked. So an example of a tool could be search, searching the web, searching a database at their disposal. Another example could be a calculator to do some math. This could be a piece of program code that maybe might manipulate the database. This can also be another language model that maybe you're trying to do a translation task and you want a model that can be able to do that. And there's so many other possibilities of what can do here. So these could be APIs, basically any piece of external program you want to give your model access to. Third capabili

Configuremos la nueva cadena usando Pinecone como almacén vectorial:

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create the prompt template
template = """Answer the following question based on the provided context:

Context:
{context}

Question:
{question}

Answer the question based on the context provided. If you cannot find the answer in the context, say so."""

prompt = ChatPromptTemplate.from_template(template)

# Create the RAG chain
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Create the chain components
chain = (
    RunnableParallel(
        context=lambda x: vectorstore.similarity_search(x["question"], k=3),
        question=RunnablePassthrough()
    )
    | prompt
    | model
    | StrOutputParser()
)

# Test the chain
question = "What are the key points discussed in the video?"
response = chain.invoke({"question": question})
print("\nQuestion:", question)
print("\nAnswer:", response)


Question: What are the key points discussed in the video?

Answer: The key points discussed in the video include the importance of system design with agentic behavior, the presence of a human in the loop for accuracy improvement, the use of tools such as search engines and calculators, the capability to access memory for storing information and retrieving it when needed, and the complexity of solving problems like determining the number of sunscreen bottles to bring on vacation.
